<a href="https://colab.research.google.com/github/yungshan629/leetcode/blob/main/%E5%9F%BA%E5%9B%A0%E6%BC%94%E7%AE%97%E6%B3%95%E6%9C%80%E4%BD%B3%E5%8C%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Setup

In [ ]:
!pip install python-dotenv finlab deap > log.txt
!pip install ta-lib-bin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 26.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from dotenv import load_dotenv
import finlab
from finlab import data
from finlab.backtest import sim
import numpy as np
import pandas as pd
finlab.login('Pqk/AHdNxUtQYWl8D0/ekfDV6xOZhihlO95F7b4Uq2VppE2kSSaHhljSpnzk9ue7#vip_m')


class SimpleReport:
    def __init__(self, report, name=None):
        self.report = report
        self.name = name
        s = self.report.get_stats()
        self.sharpe = s['daily_sharpe']
        self.sortino = s['daily_sortino']
        self.mdd = s['max_drawdown'] * 100
        self.win_ratio = s['win_ratio'] * 100
        self.annualized_return = s['cagr'] * 100
        self.calmar=s['calmar']
        self.tneagle=s['cagr']*100/abs(self.mdd)*self.sortino
        self.dailymean=s['daily_mean']*100


        self.output_template = "" # "=" * 80 + "\n"
        if self.name != None:
            self.output_template += f"Report Name: {str(self.name)}\n"
        self.output_template += "=" * 80 + "\n"
        self.output_template += f"TNeagle Ratio: {self.tneagle:.1f} | Calmar Ratio: {self.calmar:.1f} | Daily Sharpe: {self.sharpe:.1f} | Daily Sortino: {self.sortino:.1f}\n"
        self.output_template += f"Daily Mean: {self.dailymean:.1f}% | MDD: {self.mdd:.1f}% | Win Ratio: {self.win_ratio:.1f}% | CAGR: {self.annualized_return:.1f}%\n"
        self.output_template += "=" * 80 + "\n"
    def __str__(self):
        return self.output_template

輸入成功!


# Data preparation

In [ ]:
# import data from finlab
close = data.get('price:收盤價')
high = data.get("price:最高價")
low = data.get("price:最低價")
vol = data.get('price:成交股數')
rev = data.get('monthly_revenue:當月營收')
rev_yoy = data.get('monthly_revenue:去年同月增減(%)')
operating_profitM = data.get('fundamental_features:營業利益率')

netincome = data.get('fundamental_features:經常稅後淨利')
net_income_grow = data.get('fundamental_features:稅後淨利成長率')

eps = data.get('financial_statement:每股盈餘')
stock = data.get('financial_statement:存貨')
stock_turnover = data.get('fundamental_features:存貨週轉率').replace([np.nan, np.inf], 0)
season_revenue = data.get('financial_statement:營業收入淨額')
fcf_in = data.get('financial_statement:投資活動之淨現金流入_流出')
fcf_out = data.get('financial_statement:營業活動之淨現金流入_流出')


df1 = data.get('financial_statement:投資活動之淨現金流入_流出')
df2 = data.get('financial_statement:營業活動之淨現金流入_流出')
股本 = data.get('financial_statement:股本')


營業利益成長率 = data.get('fundamental_features:營業利益成長率')


權益總計 = data.get('financial_statement:股東權益總額')
融資使用率 = data.get('margin_transactions:融資使用率')
業外收支營收率 = data.get('fundamental_features:業外收支營收率')
ROE_aftertax = data.get("fundamental_features:ROE稅後")
yield_ratio = data.get('price_earning_ratio:殖利率(%)')

adj_close = data.get('etl:adj_close')
adj_high = data.get('etl:adj_high')
adj_low = data.get('etl:adj_low')

In [ ]:
# Processing

#財務指標


#技術指標


#其他

# Revenue growth
condition3 = ((rev_yoy > 0).sustain(6)) & (rev_yoy.average(6) > 25) & (rev_yoy.rise())
# Profit rate
operating_profitM_stable = (((operating_profitM.diff() / operating_profitM.shift().abs()).rolling(3).min()) * 100) > -20
operating_profitM_aa_1 = operating_profitM_stable & (operating_profitM.average(4) > 15)
operating_profitM_aa_2 = operating_profitM_stable & ((operating_profitM.average(4) <= 15) & ((operating_profitM.average(4) > 10)) & operating_profitM.rise())
condition8 = operating_profitM_aa_1 | operating_profitM_aa_2
# Profit growth
net_income_grow_aa_1 = ((net_income_grow > 0).sustain(3)) & (net_income_grow.rise())
net_income_grow_aa_2 = (net_income_grow > 50).sustain(3)
condition14 = (net_income_grow_aa_1 | net_income_grow_aa_2)

# Profit strength
condition32 = (eps.rolling(4).sum() > 5) & (eps > 0)

# Inventory turnover
stock_low = (stock_turnover <= 0) | (stock_turnover > 10) | ((stock / season_revenue) <= 0.04)
stock_turnover_stable = (stock_turnover.diff() / stock_turnover.shift().abs()).rolling(3).min() * 100 > -20
stock_turnover_cumulate_loss_gt_m20 = (stock_turnover.fall().sustain(3, 2)) & (stock_turnover.pct_change()[stock_turnover.pct_change() < 0].rolling(2).sum() * 100 < -20)
condition33 = (~stock_low) & stock_turnover_stable & (stock_turnover.average(4) > 1.5) & ~(stock_turnover_cumulate_loss_gt_m20)
# Cash flow
fcf = (fcf_in + fcf_out)
condition34 = fcf.rolling(6).min() > 0


# 近2月平均營收


# 近2月平均營收創12個月來新高
rev_ma = rev.average(2)
condition1 = rev_ma == rev_ma.rolling(12, min_periods=6).max()
# 近3月平均營收大於12月平均營收
condition12 = rev.average(3) > rev.average(12)
condition22=rev.average(3)/rev.average(12)>1.1
condition39=rev/rev.shift()>0.9


#品質因子
股東權益報酬率 = netincome/ 權益總計
當季營收 = rev.rolling(4).sum()
condition4 =(operating_profitM >= 3) & (股東權益報酬率 > -0.01)
condition28 = ROE_aftertax > 0
condition13 = operating_profitM >= 3

#高殖利率
condition11=yield_ratio>=6


# 低波動因子
atr = data.indicator('ATR', adjust_price=True,timeperiod=10)
entry_volatility = atr/adj_close # 進場訊號波動率
condition6= (entry_volatility <= 0.08)
condition39= (entry_volatility <= 0.04)
condition7= (業外收支營收率 < 5.2)
condition5 = (融資使用率 <= 34)

#量價結構

# 近一日收盤價創近2日新高
condition15 = (close == close.rolling(20).max())
# 近5日內有2日股價創新高
condition2 = (close == close.rolling(200).max()).sustain(5,2)
# 近一日收盤價創近60日新高
newhigh = close.rolling(60, min_periods=1).max() == close
condition16 = newhigh
# 前30日(不包含今日)有至少一日的收盤價未創新高
condition17 = (newhigh.shift(1) == 0).rolling(30).sum() > 0
# 第30日～第55日前的收盤價有至少一日收盤價創近60日新高
condition18 = (newhigh.shift(30).rolling(25).sum() > 0)
# 前30日～55日的收盤價最高價小於近日收盤價
condition19 = (close.shift(30).rolling(25).max() < close)
# 近日收盤價大於120前收盤價
condition20 = close > close.shift(120)
# 近日收盤價大於60前收盤價
condition21 = close > close.shift(60)
# 收盤價近5日至少有1日創收盤價近120日創新
condition29 = (close == close.rolling(120).max()).sustain(5,1)
# 近60日股價高低區間在30%內
condition30 = (1 - low.rolling(60).min()/high.rolling(60).max()) < 0.3

# 近5日平均成交量大於近20日平均成交量
condition23 = vol.average(5) > vol.average(20)



#技術指標
  #多頭排列
sma5=close.average(5)
sma20=close.average(20)
sma60=close.average(60)
sma240=close.average(240)

condition8=(close > sma5) & (sma5 > sma20)
condition9=(close > sma5) & (sma5 > sma20) & (sma20 > sma60)
condition10=(close > sma5) & (sma5 > sma20) & (sma20 > sma60) & (sma60 > sma240)

#RSI
rsi20 = data.indicator("RSI", timeperiod=20)
rsi60 = data.indicator("RSI", timeperiod=60)
rsi120 = data.indicator("RSI", timeperiod=120)

condition24 = rsi120 > 55
condition25 = rsi60 < 75
condition26 = rsi20.pct_change(3) > 0.02
condition27 = (rsi20 > 75).sustain(3)


#MACD
macd,macdsignal,macdhist = data.indicator("MACD",adjust_price=False, resample="D", fastperiod=12,slowperiod=26,signalperiod=9)
# 多方續強：其正值大於或等於前一天的正值，多方力道持續增強中。
condition31 = (macdhist > 0) & (macdhist > macdhist.shift())
# 空轉多：今天的值由負值轉為正值
condition35 = (macdhist > 0) & (macdhist.shift() < 0)
condition36= (condition31 | condition35)

#KD
k, d = data.indicator("STOCH",fastk_period=9,slowk_period=5,slowk_matype=1,slowd_period=5,slowd_matype=1)
kw, dw = data.indicator("STOCH",resample='W',fastk_period=9,slowk_period=5,slowk_matype=1,slowd_period=5,slowd_matype=1)
#KD黃金交叉
kd_crossover = (k > d) & (k.shift() < d.shift())
condition37 = (kw > dw) & (kw.shift() < dw.shift())
condition38 = kd_crossover & condition37


#布林通道
upperband, middleband, lowerband=data.indicator('BBANDS', resample='W')
condition39=close>lowerband
condition40=close<lowerband






#流動性限制
vol_ma = vol.average(5)
vol_limit = vol_ma > 200*1000

In [ ]:
#基因庫

df_list = [condition1, condition2, condition3, condition4, condition5, condition6, condition7, condition8, condition9, condition10,
       condition11 ,condition12, condition13, condition14, condition15, condition16, condition17, condition18, condition19, condition20,
       condition21, condition22, condition23, condition24, condition25, condition26, condition27, condition28, condition29, condition30,
       condition31, condition32, condition33, condition34, condition35, condition36, condition37, condition38, condition39, condition40]

# Current version

* Support cache
* Support Parallelization

* To-do:
    * Log info
    * Checkpoint storage

In [ ]:
# Evaluating function 演化函數

import pandas as pd
from functools import reduce

def eval(index_list, df_list, eval_dict):
    # Cache:
    id = ''.join([str(value) for value in index_list])
    if id in eval_dict:
        return eval_dict[id]
    else:
        # Filter the MyClass instances which are selected (index == 1)
        selected_dfs = [df for index, df in zip(index_list, df_list) if index == 1]

        # If no DataFrames are selected, return an empty DataFrame
        if not selected_dfs:
            result = pd.DataFrame()
        else:
            # Perform the intersection operation on the selected DataFrames using reduce
            result = reduce(lambda x, y: x & y, selected_dfs)

        if result.empty or result.sum().sum()<50:
            value = 0
        else:
          #回測函數，可設定停損幅度, 抽樣頻率, 最大部位等
            result=vol_limit & result
            result=result*rev_yoy
            result = result[result>0].is_largest(5).reindex(rev.index_str_to_date().index, method='ffill')
            try:
              metrics = sim(result, market='TW_STOCK', upload=False,resample='2M', stop_loss=0.08, position_limit=1/3).get_metrics()
              value = ((metrics['profitability']['annualReturn']) / abs(metrics['risk']['maxDrawdown'])) * metrics['ratio']['sortinoRatio']
            except:
              metrics=[]
              value = 0


        eval_dict[id] = (value,)
        return (value,)  # Return a tuple

In [ ]:
import pandas as pd
from functools import reduce
import random
import multiprocessing
from deap import base, creator, tools

def main(Population_size, Generation_num, Threshold, df_list):
    # Initialization
    Cond_num = len(df_list)

    # Creator
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)

    # Toolbox
    toolbox = base.Toolbox()
    # Attribute generator
    toolbox.register("attr_bool", random.randint, 0, 1)
    # Structure initializers
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, Cond_num)

    # Register the population once
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    # Operators
    num_cpus = multiprocessing.cpu_count() - 1
    pool = multiprocessing.Pool(processes=num_cpus)
    toolbox.register("map", pool.imap)

    # Use Manager to create a shared dictionary
    manager = multiprocessing.Manager()
    eval_dict = manager.dict()

    # Pass eval_dict to the evaluate function
    toolbox.register("evaluate", eval, df_list=df_list, eval_dict=eval_dict)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # Only create 10 individuals for the initial population
    pop = toolbox.population(n=10)

    # Evaluate the initial population
    fitnesses = list(toolbox.map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    CXPB, MUTPB = 0.5, 0.2
    g = 0

    # Initialize fits list
    fits = [ind.fitness.values[0] for ind in pop]

    while max(fits) < Threshold and g < Generation_num:
        g += 1
        print("-- Generation %i --" % g)

        offspring = toolbox.select(pop, len(pop))
        offspring = list(pool.imap(toolbox.clone, offspring))

        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = list(toolbox.map(toolbox.evaluate, invalid_ind))

        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        pop[:] = offspring

        fits = [ind.fitness.values[0] for ind in pop]

        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5

        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)


        # Check if max value is greater than 16 and display corresponding genes
        if max(fits)>12:
         max_index = fits.index(max(fits))
         max_genes = pop[max_index]
         print("  Genes for Max Value: %s" % max_genes)

        # For subsequent generations, set the population size to Population_size
        if g == 1:
           pop = toolbox.population(n=Population_size)

    return dict(eval_dict)

In [ ]:
# Example usage:
Population_size = 60
Generation_num = 200
Threshold = 30

result = main(Population_size, Generation_num, Threshold, df_list)

-- Generation 1 --
  Min 0.0
  Max 0.0
  Avg 0.0
  Std 0.0
-- Generation 2 --
  Min 0.0
  Max 0.12950568068793591
  Avg nan
  Std nan
-- Generation 3 --
  Min 0.0
  Max 0.12950568068793591
  Avg nan
  Std nan
-- Generation 4 --
  Min 0.0
  Max 3.042817095729478
  Avg 0.05718890229655476
  Std 0.3897182060861801
-- Generation 5 --
  Min -0.06897193453609154
  Max 3.042817095729478
  Avg 0.11759537685215184
  Std 0.5450913116225943
-- Generation 6 --
  Min 0.0
  Max 5.136345815699344
  Avg 0.2325854684293581
  Std 0.836685502067619
-- Generation 7 --
  Min -0.004661098004638658
  Max 3.042817095729478
  Avg 0.29051534641868754
  Std 0.7093098777955167
-- Generation 8 --
  Min -0.021399533685397685
  Max 3.042817095729478
  Avg nan
  Std nan
-- Generation 9 --
  Min -0.021399533685397685
  Max 3.042817095729478
  Avg nan
  Std nan
-- Generation 10 --
  Min -0.008768471855293801
  Max 5.136345815699344
  Avg nan
  Std nan
-- Generation 11 --
  Min 0.0
  Max 5.136345815699344
  Avg nan
  St

In [ ]:

# 找到前三大的值對應的標籤
sorted_items = sorted(result.items(), key=lambda x: x[1][0], reverse=True)
top_three_values = [item[1][0] for item in sorted_items[:3]]
top_three_keys = [item[0] for item in sorted_items[:3]]

print(f"前三大的值: {top_three_values}")
print(f"前三大值對應的標籤: {top_three_keys}")

前三大的值: [5.136345815699344, 3.042817095729478, 3.042817095729478]
前三大值對應的標籤: ['1101110001010000010101010111100001000000', '1100100001010001010100110111100001000000', '1101110001010001010100110111100001000000']


In [ ]:
binary_string = '00000010010100010100000010'
dist1 =['condition1', 'condition2', 'condition3', 'condition4', 'condition5', 'condition6', 'condition7', 'condition8', 'condition9', 'condition10', 'condition11', 'condition12', 'condition13', 'condition14', 'condition15', 'condition16', 'condition17', 'condition18', 'condition19', 'condition20', 'condition21', 'condition22', 'condition23', 'condition24', 'condition25', 'condition26', 'condition27', 'condition28', 'condition29', 'condition30', 'condition31', 'condition32', 'condition33', 'condition34', 'condition35', 'condition36']

# 將二進位字串轉換為布林值的列表
binary_list = [bit == '1' for bit in binary_string]

# 找出值為 True 對應的條件
active_conditions = [condition for condition, value in zip(dist1, binary_list) if value]
result_condition = '&'.join(active_conditions)

print("對應的條件：", result_condition)


對應的條件： condition7&condition10&condition12&condition16&condition18&condition25


In [ ]:


position = condition1  & condition4 & condition5 & condition7 & condition8 & condition9
position= rev_yoy*position
position = position[position>0].is_largest(8).reindex(rev.index_str_to_date().index, method='ffill')
report = sim(position, stop_loss=0.1, position_limit=1/4, name='測試１',upload=False)
report1 = sim(position, stop_loss=0.1, position_limit=1/4,trade_at_price='open_close_avg',name='測試2',upload=False)
report2 = sim(position, stop_loss=0.1, position_limit=1/4,trade_at_price='open',name='測試3',upload=False)

#report.display()


print(SimpleReport(report,'收盤買'))
print(SimpleReport(report1,'開收盤買'))
print(SimpleReport(report2,'開盤買'))


Report Name: 收盤買
TNeagle Ratio: 13.4 | Calmar Ratio: 3.5 | Daily Sharpe: 2.2 | Daily Sortino: 3.9
Daily Mean: 55.7% | MDD: -19.4% | Win Ratio: 52.5% | CAGR: 67.1%

Report Name: 開收盤買
TNeagle Ratio: 15.7 | Calmar Ratio: 3.3 | Daily Sharpe: 2.7 | Daily Sortino: 4.7
Daily Mean: 54.5% | MDD: -20.2% | Win Ratio: 51.6% | CAGR: 66.6%

Report Name: 開盤買
TNeagle Ratio: 10.3 | Calmar Ratio: 2.9 | Daily Sharpe: 2.0 | Daily Sortino: 3.5
Daily Mean: 54.3% | MDD: -21.7% | Win Ratio: 51.0% | CAGR: 64.0%

